# import required

In [ ]:
import face_recognition
from PIL import Image
import numpy as np
import tqdm
import os
from time import time

In [ ]:
face_recognition.__version__

# Make Encoding of any face image

In [ ]:
def get_encoding(img_path):
    try:
        known_image = face_recognition.load_image_file(img_path)
        face=face_recognition.face_locations(known_image,number_of_times_to_upsample=2)
        known_encoding = face_recognition.face_encodings(known_image,known_face_locations=face,num_jitters=4)[0]
        return known_encoding
    except:
        return None

# mongo connection

In [ ]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["face_database"]

mycol = mydb["face_collection"]

## check if collection exists

In [ ]:
collist = mydb.list_collection_names()
if "face_collection" in collist:
      print("The collection exists.")

In [ ]:
def insert_record_into_collection(request_id, client_id,encoding,matched_ids=[]):
    data={'request_id':str(request_id), 'client_id':str(client_id),'matched_ids':[str(i) for i in matched_ids]}
    encoding_data=dict([(f'e{idx+1}',i) for idx,i in enumerate(encoding)])
#     data['encodings']=list(encoding)
    data['encodings']=encoding_data
    status=mycol.insert_one(data)
    return status

In [ ]:
def compare_from_collection(encoding,client_id,threshold=0.55):
    step1=dict([(f't_e{idx+1}',{'$pow':[{'$subtract':[f'$encodings.e{idx+1}', i]}, 2]}) for idx,i in enumerate(encoding) ])
    data_to_return={'request_id': 1, 
            'client_id': 1, 
            'matched_ids': 1, 
            'encodings': 1}
    step1.update(data_to_return)
    result=mycol.aggregate([
         {
        '$project': data_to_return
            },
         {
            '$match': {
                'client_id': str(client_id)
            }
        },
        {
            '$project': step1
        }, 
        {
            '$project': {
                'request_id': 1, 
            'client_id': 1, 
            'matched_ids': 1, 
                'distance': {
                    '$sqrt': {
                        '$sum':[f'$t_e{i}' for i in range(1,129)]
                    }
                }
            }
        },
         {
        '$match': {
            'distance': {
                '$lt': threshold
            }
        }
    }
    ]
    )
    return list(result)

## make image encoding

In [ ]:
img_path = './haseeb.jpg'
known_encoding=get_encoding(img_path)
if type(known_encoding)==np.ndarray:
    print(len(known_encoding))
    display(Image.open(img_path).resize((200,200)))
else:
    print('no_encoding_find!')
    display(Image.open(img_path).resize((200,200)))

## insert record

In [ ]:
insert_record_into_collection(request_id=2002, client_id=2,matched_ids=[], encoding=known_encoding)

## make encoding 

In [ ]:
img_path = './talat.jpg'
known_encoding=get_encoding(img_path)
if type(known_encoding)==np.ndarray:
    print(len(known_encoding))
    display(Image.open(img_path).resize((200,200)))
else:
    print('no_encoding_find!')
    display(Image.open(img_path).resize((200,200)))

## compare encoding with db encodings

In [ ]:
start=time()
display(compare_from_collection(encoding=known_encoding,client_id='test',threshold=0.47))
time()-start

# Insert fake records

In [ ]:
def insertion(i):
    insert_record_into_collection(request_id=i, client_id="test", encoding=np.random.random_sample(128),matched_ids=[])


In [ ]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

with ThreadPoolExecutor(max_workers = 50) as executor:
    executor.map(insertion, list(range(50000,1000000)))


In [ ]:
a=get_encoding('./saleem.jpg')
b=get_encoding('./4.jpg')

In [ ]:
np.linalg.norm(a-[b])

In [ ]:
client_id=1
encoding=range(1,129)
[
     {
        '$match': {
            'client_id': client_id
        }
    },
    {
        '$project': dict([(f't_e{idx+1}',{'$pow':[{'$subtract':[f'$encodings.e{idx+1}', i]}, 2]}) for idx,i in enumerate(encoding) ])
    }, 
    {
        '$project': {
            'total': {
                '$sqrt': {
                    '$sum':[f'$t_e{i}' for i in range(1,129)]
                }
            }
        }
    }
]


In [ ]:
dict([(f't_e{idx}',{'$pow':[{'$subtract':[f'$encodings.{idx}1', i]}, 2]}) for idx,i in enumerate(range(1,129)) ])

In [ ]:
 [
                        '$t_e1', '$t_e2'
                    ]
    
[f'$t_e{i}' for i in range(1,129)]

In [ ]:
import numpy as np

def distance(arr1,arr2):    
    return np.sum((arr1 - arr2)**2)**0.5

In [ ]:
p=np.array([1,2,1,4,5,3,7,8,9,10])
q=np.array([1,2,3,4,5,6,7,8,9,10])

In [ ]:
distance(p,q)

In [ ]:
a=np.sqrt(sum([(i-j)**2 for i,j in zip(p,[10]*len(p))]))
a

In [ ]:
b=np.sqrt(sum([(i-j)**2 for i,j in zip(q,[10]*len(q))]))
b

In [ ]:
abs(a-b)